In [1]:
from my_weapon import *
from tqdm import tqdm_notebook as tqdm
from fake_identify import Are_you_IRA
import matplotlib
from collections import Counter
matplotlib.rcParams["font.size"] = 14
sns.set_style("darkgrid")
ira_c = sns.color_palette("coolwarm", 8)[7]
all_c = sns.color_palette("coolwarm", 8)[0]

Putin = Are_you_IRA()

In [2]:
import re
import numpy as np
import pandas as pd
from pprint import pprint

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# spacy for lemmatization
import spacy
nlp = spacy.load('en', disable=['parser', 'ner'])

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

In [3]:
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend([
    "rt", "…", "...", "URL", "http", "https", "“", "”", "‘", "’", "get", "2", "new", "one", "i'm", "make",
    "go", "good", "say", "says", "know", "day", "..", "take", "got", "1", "going", "4", "3", "two", "n",
    "like", "via", "u", "would", "still", "first", "really", "watch", "see", "even", "that's", "look", "way",
    "last", "said", "let", "twitter", "ever", "always", "another", "many", "things", "may", "big", "come", "keep",
    "5", "time", "much", "want", "think", "us", "love", "people", "need"
])

In [4]:
from Trump_Clinton_Classifer.TwProcess import CustomTweetTokenizer

tokenizer = CustomTweetTokenizer(preserve_case=False,
                                 reduce_len=True,
                                 strip_handles=False,
                                 normalize_usernames=False,
                                 normalize_urls=True,
                                 keep_allupper=False)

In [5]:
texts = []

data = pd.read_csv("data/ira-tweets-ele.csv", usecols=["tweet_text"])
for i, row in tqdm(data.iterrows()):
    # if row["userid"] not in users_comm:
    #     continue
    words = tokenizer.tokenize(row["tweet_text"])
    words = [w for w in words if w not in stop_words and w]
    # if words[0] == "RT":
    #     continue
#     for w in words:
#         cnt[w] += 1
    texts.append(words)
print(len(texts))
# json.dump(cnt.most_common(), open("data/word_cloud.json", "w"), indent=2)


391680


In [74]:
# Define functions for stopwords, bigrams, trigrams and lemmatization
# nlp = spacy.load('en')
import re

def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]


def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV', 'PROPN']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    i = 0
    for sent in tqdm(texts):
        sent = " ".join(sent)
        sent = re.sub(r'#(\w+)', r'itstopiczzz\1', sent)
        sent = re.sub(r'@(\w+)', r'itsmentionzzz\1', sent)
        doc = nlp(sent)
        
        _d = [token.lemma_ for token in doc if token.pos_ in allowed_postags and token.lemma_ not in stop_words]
#         _d = [(token.pos_, token.lemma_) for token in doc if token.lemma_ not in stop_words]
        
        _d = [x.replace('itstopiczzz', '#') for x in _d]
        _d = [x.replace('itsmentionzzz', '@') for x in _d]
        texts_out.append(_d)

    return texts_out

In [75]:
texts_out = lemmatization(texts)

In [78]:
len(texts_out)

391680

In [81]:
# Create Dictionary
id2word = corpora.Dictionary(texts_out)

# Create Corpus
texts = texts_out

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[:1])

[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1)]]


In [ ]:
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus, id2word=id2word, num_topics=6)

In [ ]:
print(corpus[:1])